- start with ODEs and Michaelis-Menten form of growth curves.
- parameter used from [Kim et al 2005](https://doi.org/10.1007/s00253-005-1997-z), and [Reardon et al 2000](https://doi.org/10.1002/1097-0290(20000820)69:4<385::AID-BIT5>3.0.CO;2-Q)

### basic models



---
### key parameters



In [5]:
# packages


# parameters


In [6]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from scipy.stats import linregress

# Define the system of ODEs
def basic_growth_model(y, t, params):
    X, S = y  # Biomass and substrate concentrations
    
    # Unpacking parameters
    mu_max, Ks = params  
    
    # Monod equation (basic form)
    mu = mu_max * (S / (S + Ks))
    
    # Biomass growth rate
    dXdt = mu * X
    
    # Substrate consumption rate (not included in basic form, but can be added later)
    dSdt = 0  # Placeholder if needed later
    
    return [dXdt, dSdt]

# Function to estimate mu_max from OD600 data
def estimate_mu_max(time, od_values):
    log_od = np.log(od_values)
    max_slope = 0
    best_fit = (0, 0)
    
    for i in range(len(time) - 5):  # Sliding window for linear regression
        slope, intercept, _, _, _ = linregress(time[i:i+5], log_od[i:i+5])
        if slope > max_slope:
            max_slope = slope
            best_fit = (slope, intercept)
    
    return best_fit[0]  # mu_max is the slope of the best-fit line

# Load OD600 data from an Excel file
file_path = "YM-25/Mres_DoL_Modelling/OD600_modelling/try.xlsx"  # Update with actual file path
sheet_name = 0  # Modify if using a specific sheet
data = pd.read_excel(file_path, sheet_name=sheet_name)

time_data = data["Time (s)"]  # Time in seconds
od_data = data["OD600"]       # OD600 readings

# Estimate mu_max
mu_max_estimated = estimate_mu_max(time_data, od_data)
print(f"Estimated mu_max: {mu_max_estimated:.4f} per second")

# Plot results
plt.figure(figsize=(8, 5))
plt.plot(time_data, np.log(od_data), marker='o', linestyle='-', label='Log OD600')
plt.xlabel('Time (seconds)')
plt.ylabel('ln(OD600)')
plt.title('Estimating Maximum Growth Rate')
plt.legend()
plt.show()

# Use estimated mu_max in model
X0 = 0.01  # Initial biomass concentration
S0 = 1.0   # Initial substrate concentration
y0 = [X0, S0]
Ks = 0.2    # Half-saturation constant (g/L)
params = (mu_max_estimated, Ks)

# Time vector for simulation
t = np.linspace(0, max(time_data), 500)

# Solve ODEs
solution = odeint(basic_growth_model, y0, t, args=(params,))
X_sol, S_sol = solution.T

# Plot simulated growth curve
plt.figure(figsize=(8, 5))
plt.plot(t, X_sol, label='Biomass (X) Simulated', linewidth=2)
plt.xlabel('Time (seconds)')
plt.ylabel('Concentration (g/L)')
plt.legend()
plt.title('Basic Microbial Growth Model with Estimated mu_max')
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'YM-25/Mres_DoL_Modelling/OD600_modelling/try.xlsx'